## モデルの確認

In [2]:
import ollama

# Ollamaサーバーから利用可能なモデルのリストを取得します
# response_objectにはモデル情報を含むオブジェクトが返されます
response_object = ollama.list()

# 'models'属性があり、かつモデルリストが空でないかを確認します
if hasattr(response_object, 'models') and response_object.models:
    # 利用可能なモデル名のみを出力します
    for model_data in response_object.models:
        # model_dataオブジェクトからモデル名（.model属性）を取得して表示します
        print(model_data.model)
else:
    # モデルが見つからなかった場合のメッセージ
    print("利用可能なモデルが見つかりませんでした。Ollamaサーバーが起動しており、モデルがインストールされているか確認してください。")

llama2:13b
qwen2.5-coder:32b


## 出力サンプル

In [ ]:
import ollama

# --- 設定項目 ---
# 使用するモデル名を指定します (1セル目で確認したモデル名を入力してください)
MODEL_TO_USE = 'deepseek-r1:8b'  # 例: 'gemma3:1b', 'llama3'など

# LLMへの指示（プロンプト）
USER_PROMPT = "あなたのできることを教えて下さい"

# (オプション) システムプロンプトでAIの基本的な振る舞いを指示できます
# ここでは非常にシンプルな指示にしています
SYSTEM_PROMPT = "あなたはAIアシスタントです。簡潔に答えてください。"
# --- 設定ここまで ---

# LLMにチャット形式で指示を送信します
response = ollama.chat(
    model=MODEL_TO_USE,
    messages=[
        {
            'role': 'system',
            'content': SYSTEM_PROMPT
        },
        {
            'role': 'user',
            'content': USER_PROMPT
        }
    ]
)

# LLMからの応答メッセージの内容のみを出力します
print(response['message']['content'])

## 応答速度比較

In [3]:
# deepseek-r1:8b, llama3.1:8b, qwen2.5-coder:7b のそれぞれの応答速度を確かめる
import time
import ollama

# --- 設定 ---
MODELS = [
    "llama2:13b",
    "qwen2.5-coder:32b"
]
SYSTEM_PROMPT = "あなたはAIアシスタントです。簡潔に答えてください。"
USER_PROMPT = "あなたのできることを教えて下さい"
RUNS_PER_MODEL = 3  # 1モデルあたり何回計測するか
# --- 設定ここまで ---

def measure_model(model_name):
    times = []
    answers = []
    for i in range(RUNS_PER_MODEL):
        start = time.time()
        response = ollama.chat(
            model=model_name,
            messages=[
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user',   'content': USER_PROMPT},
            ]
        )
        elapsed = time.time() - start
        times.append(elapsed)
        answers.append(response['message']['content'])
        print(f"[{model_name} Run {i+1}] {elapsed:.3f} 秒")
    avg_time = sum(times) / len(times)
    print(f"→ {model_name} の平均応答時間: {avg_time:.3f} 秒\n")
    return {
        'model': model_name,
        'times': times,
        'avg_time': avg_time,
        'answers': answers,
    }

def main():
    results = []
    print("=== 応答速度 & 回答内容 比較 ===\n")
    for m in MODELS:
        print(f"--- モデル: {m} ---")
        try:
            res = measure_model(m)
            results.append(res)
        except Exception as e:
            print(f"Error for {m}: {e}\n")
    # 結果をまとめて表示
    print("=== まとめ ===")
    for r in results:
        print(f"\nモデル: {r['model']}")
        print(f"平均時間: {r['avg_time']:.3f} 秒")
        print("【回答例】")
        for idx, ans in enumerate(r['answers'], 1):
            print(f"  Run {idx}: {ans[:100]}{'…' if len(ans)>100 else ''}")
    # 必要に応じてここで自動評価指標を計算してください

if __name__ == '__main__':
    main()



=== 応答速度 & 回答内容 比較 ===

--- モデル: llama2:13b ---
[llama2:13b Run 1] 19.306 秒
[llama2:13b Run 2] 16.740 秒
[llama2:13b Run 3] 14.925 秒
→ llama2:13b の平均応答時間: 16.991 秒

--- モデル: qwen2.5-coder:32b ---
[qwen2.5-coder:32b Run 1] 14.355 秒
[qwen2.5-coder:32b Run 2] 7.048 秒
[qwen2.5-coder:32b Run 3] 6.401 秒
→ qwen2.5-coder:32b の平均応答時間: 9.268 秒

=== まとめ ===

モデル: llama2:13b
平均時間: 16.991 秒
【回答例】
  Run 1: 
Hello! As an AI assistant, I can assist with a wide range of tasks. Here are some examples of what …
  Run 2: 
Hello! As an AI assistant, I can assist with a variety of tasks. Here are some examples of what I c…
  Run 3: 
 Hello! As an AI assistant, I can help with a wide range of tasks. Here are some examples of what I…

モデル: qwen2.5-coder:32b
平均時間: 9.268 秒
【回答例】
  Run 1: 情報検索、要約作成、翻訳、スケジュール管理などを行えます。
  Run 2: 情報検索、要約作成、翻訳、予定管理などを行います。
  Run 3: 情報を検索、質問に回答、スケジュール管理などを行います。


### 応答速度比較結果

| モデル                | Run 1 (秒) | Run 2 (秒) | Run 3 (秒) | 平均応答時間 (秒) |
|:---------------------|-----------:|-----------:|-----------:|-----------------:|
| deepseek-r1:8b       |     58.117 |    140.283 |    126.096 |           108.165 |
| llama3.1:8b          |     24.420 |     15.763 |     17.661 |            19.281 |
| qwen2.5-coder:7b     |     37.433 |     26.084 |     21.571 |            28.363 |


### desktop 環境

モデル: llama2:13b

| Run 1 (秒) | Run 2 (秒) | Run 3 (秒) | 平均応答時間 (秒) |
|:-----------|-----------|-----------|-----------------|
| 19.306 | 16.740 | 14.925 | 16.991 |

モデル: qwen2.5-coder:32b

| Run 1 (秒) | Run 2 (秒) | Run 3 (秒) | 平均応答時間 (秒) |
|:-----------|-----------|-----------|-----------------|
| 14.355 | 7.048 | 6.401 | 9.268 |